In [1]:
from base_app.models import ConfigItems
from leagues.models import Competition, Season

In [2]:
from seleniumwire import webdriver
from selenium.webdriver.common.by import By
import requests,os,sys,json
from bs4 import BeautifulSoup
from time import sleep
from pprint import pprint
import pandas as pd

In [ ]:
config_obj = {conf.key : conf.value for conf in ConfigItems.objects.all()}

In [ ]:
available_countries = Competition.objects.values_list('country', flat=True).distinct()

available_comps_scores_away_items = Competition.objects.values_list('country','name_scoresaway').distinct()
available_comps_scores_away = {}
for country,competition in available_comps_scores_away_items :
    if country not in available_comps_scores_away :
        available_comps_scores_away[country]  = []
    available_comps_scores_away[country].append(competition.lower())
    
available_comps_fotmob_items = Competition.objects.filter(shot_data_available=True).values_list('country','name_fotmob').distinct()
lowercase_names,available_comps_fotmob,lowercase_comps = [],{},[]
for country,competition in available_comps_fotmob_items :
    if country not in available_comps_fotmob :
        available_comps_fotmob[country]  = []
    available_comps_fotmob[country].append(competition.lower())
    lowercase_names.append(str(country).lower())
    lowercase_comps.append(str(competition).lower())


In [ ]:
driver = webdriver.Chrome()
driver.get(config_obj.get('SCORESAWAY_COMPETETIONS_URL',None))


In [ ]:
all_country_divs = driver.find_elements(By.CSS_SELECTOR, ".country.closed")
data = []
country_data = {}
for div in all_country_divs:
    country_name = div.find_element(By.CSS_SELECTOR, "span.name").get_attribute('innerText')
    if country_name in available_countries:
        comps = {comp.get_attribute('innerText'): comp.find_element(By.CSS_SELECTOR, 'a').get_property('href') for comp in div.find_elements(By.CSS_SELECTOR, "li.comp")}
        valid_comps = {comp:link for comp,link in comps.items() if comp.lower() in available_comps_scores_away[country_name]}
        country_data[country_name] = 0
        for v_comp,comp_url in valid_comps.items():
            data.append({
                "country" : country_name,
                "comp" : v_comp,
                "url" : comp_url
            })
            country_data[country_name]+=1    

In [ ]:
found_counts = dict(sorted(country_data.items()))

In [ ]:
db_counts = dict(sorted({k:len(v) for k,v in available_comps_scores_away.items()}.items()))

In [ ]:
for x in db_counts.keys() :
    if found_counts.get(x,-1) != db_counts.get(x,-1) :
        print(f"For {x} , Found {found_counts.get(x)}, in DB {db_counts.get(x)}")

In [ ]:
# pd.DataFrame(data).to_excel("Temp_data.xlsx",index=False)

In [ ]:
(sum(db_counts.values())),sum(found_counts.values()),len(available_comps_scores_away_items)

In [ ]:
for row in data:
    comp_obj = Competition.objects.get(country=row['country'],name_scoresaway = row['comp'])
    comp_obj.event_data_url = row.get('url',"PLACEHOLDER")
    # comp_obj.save()

# FOR FOTMOB

In [ ]:
with open("./all_comps_fotmob.json","r") as f :
    data = json.loads(f.read())

In [ ]:
int_comps = data.get('international',[{}])[0] 
int_comps = int_comps.get('leagues')

In [ ]:
national_leagues = data.get('countries')

In [ ]:
valid_nations = [x for x in national_leagues if x.get('name').lower() in lowercase_names ]

In [ ]:
set(lowercase_names) - set([x['name'].lower() for x in valid_nations])

#^^ DO MANUSALLY ^^^

In [ ]:
return_data = [] # name_conf
for country in valid_nations:
    avail_comps = available_comps_fotmob[country.get('name')]
    for l in country.get('leagues',[]) :
        temp = {}
        print(f"league : {l.get('name')}  | avail_comps : {avail_comps}")
        if l.get('name').lower() in avail_comps :
            temp['country'] = country.get('name')
            temp['comp'] = l.get('name')
            temp['url'] = l.get('pageUrl')
            return_data.append(temp)
            print()

In [ ]:
return_data

In [ ]:
df = pd.read_excel("Temp_data.xlsx",sheet_name="fotmob")

In [ ]:
for i,row in df.iterrows():
    try:
        comp_item = Competition.objects.get(
                    confederation = row.get('confederation'),
                    country = row.get('country'),
                    competition_name = row.get('competition_name')
        )
        comp_item.name_fotmob = row.get('fotmob_name','')
        comp_item.shot_data_url = f"https://www.fotmob.com{row.get('page_url','')}"
        comp_item.save()
    except Exception as e :
        print("Issue for ",row,e)

In [ ]:
df.head(2)

In [3]:
df = pd.read_excel("Temp_data.xlsx",sheet_name="scoresaway")

In [4]:
df

,country,comp,url
0,AFC,AFC Champions League Elite,https://www.scoresway.com/en_GB/soccer/afc-cha...
1,AFC,AFC Champions League Two,https://www.scoresway.com/en_GB/soccer/afc-cha...
2,AFC,AFC Challenge Cup,https://www.scoresway.com/en_GB/soccer/afc-cha...
3,Australia,A-League Men,https://www.scoresway.com/en_GB/soccer/a-leagu...
4,India,Indian Super League,https://www.scoresway.com/en_GB/soccer/indian-...
...,...,...,...
67,Colombia,Primera A,https://www.scoresway.com/en_GB/soccer/primera...
68,Ecuador,Liga Pro,https://www.scoresway.com/en_GB/soccer/liga-pr...
69,Paraguay,Division Profesional,https://www.scoresway.com/en_GB/soccer/divisio...
70,Peru,Liga 1,https://www.scoresway.com/en_GB/soccer/liga-1-...


In [11]:
for i,row in df.iterrows():
    try:
        comp_item = Competition.objects.get(
                    country = row.get('country'),
                    name_scoresaway = row.get('comp')
        )
        comp_item.event_data_url = row.get('url','')
        comp_item.save()
    except Exception as e :
        print("Issue for ",row,e)